In [1]:
import pandas as pd
import os
import laspy
import numpy as np
import open3d as o3d
import util_las as las
import pathlib
import geopandas as gpd
from shapely.geometry import Point

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


This file is intenteded to produce the change detections in the desired file format. Run the first cells to load the proper .csv file and then the cells for the format you want to save the detections to (.las, subset of points to .las, shapefile or .ply mesh)

In [2]:
WORKING_DIR = '/mnt/data-01/nmunger/proj-qalidar/data' 

# Output directory
folder_dir = 'out_dataframe/criticity_changes_df/'
csv_file_name = '2547000_1211500_150_2301-1541.csv'
out_dir = 'out_vis/'

tile_decript_name = csv_file_name.split('.')[0]
vox_dimension = float(tile_decript_name.rsplit('_', maxsplit=2)[1])/100

os.chdir(WORKING_DIR)

In [3]:
subfolder_path = os.path.join(out_dir,tile_decript_name)

In [4]:
# Create the path for the folder and subfolder to store the outgoing file in case it doesn't yet exist
pathlib.Path(subfolder_path).mkdir(parents=True, exist_ok=True)

### Import .csv file

In [5]:
df = pd.read_csv(os.path.join(folder_dir, csv_file_name))

In [6]:
df.head()

,X_grid,Y_grid,Z_grid,1_prev,2_prev,3_prev,6_prev,7_prev,17_prev,1_new,...,6_new,7_new,17_new,vox_id,change_criticity,cosine_similarity,second_cosine_similarity,third_cosine_similarity,majority_class,change_criticity_label
0,2547000.75,1211500.75,933.25,0.0,0.0,0.0,15.0,0.0,0.0,0.0,...,214.0,0.0,0.0,0,non_prob,1.000000,NaN,NaN,6_new,1.0
1,2547000.75,1211502.25,925.75,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,1,grey_zone,0.986394,1.0,0.986394,2_new,8.0
2,2547000.75,1211502.25,928.75,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,17.0,0.0,0.0,2,non_prob,1.000000,NaN,NaN,6_new,1.0
3,2547000.75,1211502.25,930.25,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,20.0,0.0,0.0,3,non_prob,1.000000,NaN,NaN,6_new,1.0
4,2547000.75,1211502.25,931.75,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,20.0,0.0,0.0,4,non_prob,1.000000,NaN,NaN,6_new,1.0


### Create a subset of the dataframe

In [7]:
subset_df = df.groupby('change_criticity_label').apply(lambda x: x.sample(8, random_state=42).reset_index(drop=True)).reset_index(drop=True)

subset_df.head(10)

,X_grid,Y_grid,Z_grid,1_prev,2_prev,3_prev,6_prev,7_prev,17_prev,1_new,...,6_new,7_new,17_new,vox_id,change_criticity,cosine_similarity,second_cosine_similarity,third_cosine_similarity,majority_class,change_criticity_label
0,2547278.25,1211829.25,924.25,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,145361,non_prob,1.000000,NaN,NaN,3_prev,1.0
1,2547050.25,1211577.25,921.25,0.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25827,non_prob,1.000000,NaN,NaN,2_new,1.0
2,2547264.75,1211994.25,943.75,0.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,138563,non_prob,1.000000,NaN,NaN,2_new,1.0
3,2547486.75,1211718.25,960.25,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,288563,non_prob,1.000000,NaN,NaN,3_new,1.0
4,2547249.75,1211646.25,921.25,0.0,18.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,130140,non_prob,1.000000,NaN,NaN,2_new,1.0
5,2547440.25,1211860.75,925.75,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,239587,non_prob,1.000000,NaN,NaN,3_new,1.0
6,2547248.25,1211560.75,931.75,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,1.0,0.0,0.0,129202,non_prob,1.000000,NaN,NaN,6_prev,1.0
7,2547492.75,1211767.75,958.75,0.0,0.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,295421,non_prob,1.000000,NaN,NaN,3_new,1.0
8,2547462.75,1211755.75,939.25,0.0,12.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,263545,non_prob,0.932966,NaN,NaN,2_new,2.0
9,2547462.75,1211713.75,954.25,0.0,19.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,263327,non_prob,0.998859,NaN,NaN,2_new,2.0


### Save to .las file

In [8]:
las_file = las.df_to_las(df)

las_file.write(os.path.join(subfolder_path, 'change_detection.las'))

### Save subset of the changes detection to .las and to .csv

In [ ]:
las_file = las.df_to_las(subset_df, index_to_point_source_id=True)

las_file.write(os.path.join(subfolder_path, 'subset_change_detections.las'))

In [ ]:
subset_df.to_csv(os.path.join(subfolder_path, 'subset_change_detections.csv'))

### DBSCAN clustering for isolated voxels removal -> to .las

In [ ]:
problematic_df = df[df.change_criticity=='problematic']

In [ ]:
from sklearn.cluster import DBSCAN

X = problematic_df[['X_grid','Y_grid','Z_grid']]
clustering = DBSCAN(eps=1.5, min_samples=2).fit(X)
isolated_voxel_mask = (clustering.labels_ == -1)

In [ ]:
labels_criticity=df['change_criticity_label'].values.copy()

# Change all voxels that are problematic but isolated to label 14
labels_criticity[problematic_df.index[isolated_voxel_mask]] = 14
df.loc[:, 'filtered_change_criticity_label'] = labels_criticity

In [ ]:
las_file = las.df_to_las(df, user_data_col='filtered_change_criticity_label')

las_file.write(os.path.join(subfolder_path, 'change_detection_filtered.las'))

### Save subset to geopackage

In [ ]:
geometry = [Point(xyz) for xyz in zip(subset_df.X_grid, subset_df.Y_grid, subset_df.Z_grid)]
gdf = gpd.GeoDataFrame(subset_df, crs='EPSG:2056', geometry=geometry)
gdf['geometry'] = gdf.geometry #.buffer(vox_dimension/2, cap_style=3) # Uncomment to have the shape representing the dimension of the voxel
#Remove unnecessary columns
gdf.drop(columns=['X_grid','Y_grid','Z_grid','cosine_similarity','second_cosine_similarity','third_cosine_similarity','majority_class'],inplace=True)
gdf = gdf.rename_axis('vox_id').reset_index()
gdf['validity']=1
gdf['comment']=''

In [ ]:
gdf.head()

In [ ]:
gdf.to_file(os.path.join(subfolder_path,'subset_change_detections.gpkg'), drive='GPKG')

### Save to shapefile
Not used currently, partially old code, left for legacy

In [ ]:
problematic_and_grey_df = df[df['change_criticity']!='non_prob']

In [ ]:
problematic_and_grey_df.loc[:,'Z_grid'] = problematic_and_grey_df['Z_grid'].astype(str)
problematic_and_grey_df.loc[:,'change_criticity_label'] = problematic_and_grey_df['change_criticity_label'].astype(str)

In [ ]:
problematic_and_grey_df.loc[:,'vertical_descript'] = problematic_and_grey_df[['Z_grid', 'change_criticity', 'change_criticity_label']].agg('-'.join, axis=1)
problematic_and_grey_df.head()

In [ ]:
problematic_df = problematic_and_grey_df[problematic_and_grey_df['change_criticity']=='problematic']
grey_zone_df = problematic_and_grey_df[problematic_and_grey_df['change_criticity']=='grey_zone']

In [ ]:
# Code inspired from https://stackoverflow.com/questions/17841149/pandas-groupby-how-to-get-a-union-of-strings
# We want to group all the voxels having the same planar coordinates, and create a string with every vertical descript
def f(x):
    return pd.Series(dict( vertical_descript = "{%s}" % '\n'.join(x['vertical_descript'])))

In [ ]:
grouped_problematic_df = problematic_df.groupby(['X_grid','Y_grid']).apply(f).reset_index()

In [ ]:
grouped_grey_zone_df = grey_zone_df.groupby(['X_grid','Y_grid']).apply(f).reset_index()

In [ ]:
geometry = [Point(xy) for xy in zip(grouped_problematic_df.X_grid, grouped_problematic_df.Y_grid)]
gdf_problematic = gpd.GeoDataFrame(grouped_problematic_df, crs='EPSG:2056',geometry=geometry)
gdf_problematic['geometry'] = gdf_problematic.geometry.buffer(voxel_dimension/2, cap_style=3)

In [ ]:
geometry = [Point(xy) for xy in zip(grouped_grey_zone_df.X_grid, grouped_grey_zone_df.Y_grid)]
gdf_grey_zone = gpd.GeoDataFrame(grouped_grey_zone_df, crs='EPSG:2056',geometry=geometry)
gdf_grey_zone['geometry'] = gdf_grey_zone.geometry.buffer(voxel_dimension/2, cap_style=3)

In [ ]:
# Test without concatenating the shapes
geometry = [Point(xyz) for xyz in zip(problematic_df.X_grid, problematic_df.Y_grid, problematic_df.Z_grid)]
gdf_complete_problematic = gpd.GeoDataFrame(problematic_df[['X_grid','Y_grid','Z_grid','change_criticity','change_criticity_label','vertical_descript']], crs='EPSG:2056', geometry=geometry)
gdf_complete_problematic['valid_detection']=1
gdf_complete_problematic['geometry'] = gdf_complete_problematic.geometry.buffer(voxel_dimension/2, cap_style=3)

In [ ]:
geometry = [Point(xyz) for xyz in zip(grey_zone_df.X_grid, grey_zone_df.Y_grid, grey_zone_df.Z_grid)]
gdf_complete_grey_zone = gpd.GeoDataFrame(grey_zone_df[['X_grid','Y_grid','Z_grid','change_criticity','change_criticity_label','vertical_descript']], crs='EPSG:2056',geometry=geometry)
gdf_complete_grey_zone['valid_detection']=1
gdf_complete_grey_zone['geometry'] = gdf_complete_grey_zone.geometry.buffer(voxel_dimension/2, cap_style=3)

In [ ]:
gdf_complete_problematic.to_file(f'/mnt/data-01/nmunger/out_shapefile/problematic_without_concat_{tile_name}_{voxel_dimension}.shp')

In [ ]:
gdf_complete_grey_zone.to_file(f'/mnt/data-01/nmunger/out_shapefile/grey_zone_without_concat_{tile_name}_{voxel_dimension}.shp')

In [ ]:
gdf_problematic.to_file(f'/mnt/data-01/nmunger/out_shapefile/problematic_{tile_name}_{voxel_dimension}.shp')

In [ ]:
gdf_grey_zone.to_file(f'/mnt/data-01/nmunger/out_shapefile/grey_zone_{tile_name}_{voxel_dimension}.shp')

### Save to voxel mesh

In [ ]:
def generate_voxels_mesh(voxels_center, voxel_width, voxel_height):
    '''voxels_center: dataframe of size nx3, X|Y|Z '''
    for i in range(len(voxels_center)):
        center = voxels_center.iloc[i,:3].to_numpy()
        
        # Create the cube mesh
        cube_mesh = o3d.geometry.TriangleMesh.create_box(width=voxel_width, height=voxel_height, depth=voxel_width)

        # Translate the cube to the desired center point
        cube_mesh.translate(center-np.array([voxel_width/2, voxel_width/2, voxel_height/2]))

        if i == 0: # If generating first voxel mesh
            total_mesh=cube_mesh
            continue
        else:
            total_mesh+=cube_mesh
    
    return total_mesh

In [ ]:
for change_type in df.change_criticity.unique():
    voxel_mesh = generate_voxels_mesh( df.loc[df['change_criticity']==change_type, ['X_grid','Y_grid','Z_grid']], vox_dimension, vox_dimension)
    o3d.io.write_triangle_mesh(os.path.join(subfolder_path,f'{change_type}.ply'), voxel_mesh, write_vertex_colors=True)